# АЛГОРИТМ ECLAT

Алгоритм ECLAT расшифровывается как кластеризация классов эквивалентности и обход решетки снизу вверх . Это один из популярных методов объединения правил майнинга . Это более эффективная и масштабируемая версия алгоритма Apriori. Хотя алгоритм Apriori работает в горизонтальном смысле, имитируя поиск графика по ширине, алгоритм ECLAT работает по вертикали, как поиск графика по глубине. Этот вертикальный подход алгоритма ECLAT делает его более быстрым алгоритмом, чем алгоритм Apriori.

### Как работает алгоритм?

Основная идея состоит в том, чтобы использовать пересечения наборов идентификаторов транзакций (tidsets) для вычисления значения поддержки кандидата и избегать генерации подмножеств, которые не существуют в дереве префиксов. При первом вызове функции все отдельные элементы используются вместе с их наборами приливов. Затем функция вызывается рекурсивно, и в каждом рекурсивном вызове каждая пара элемент-тидсет проверяется и объединяется с другими парами элемент-тидсет. Этот процесс продолжается до тех пор, пока не будут объединены пары-кандидаты-тидсет.

Давайте теперь разберемся с изложенным выше, работая с примером:

In [6]:
# подгрузим модули
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

In [14]:
# загрузим данные
dataset = pd.read_csv('./csvs/Online_Retail.csv', sep='\t', comment='#', header = None, delimiter = ';')

C:\Users\irina\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
dataset.head(20)

,0,1,2,3,4,5,6,7
0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01 12 10 8:26,"2,55",17850,United Kingdom
2,536365,71053,WHITE METAL LANTERN,6,01 12 10 8:26,"3,39",17850,United Kingdom
3,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01 12 10 8:26,"2,75",17850,United Kingdom
4,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01 12 10 8:26,"3,39",17850,United Kingdom
5,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01 12 10 8:26,"3,39",17850,United Kingdom
6,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,01 12 10 8:26,"7,65",17850,United Kingdom
7,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,01 12 10 8:26,"4,25",17850,United Kingdom
8,536366,22633,HAND WARMER UNION JACK,6,01 12 10 8:28,"1,85",17850,United Kingdom
9,536366,22632,HAND WARMER RED POLKA DOT,6,01 12 10 8:28,"1,85",17850,United Kingdom


In [16]:
dataset.fillna(method = 'ffill',axis = 1, inplace = True)

In [17]:
dataset.head()

,0,1,2,3,4,5,6,7
0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01 12 10 8:26,"2,55",17850,United Kingdom
2,536365,71053,WHITE METAL LANTERN,6,01 12 10 8:26,"3,39",17850,United Kingdom
3,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01 12 10 8:26,"2,75",17850,United Kingdom
4,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01 12 10 8:26,"3,39",17850,United Kingdom


In [20]:
transactions = []
for i in range(0, 7000): 
    transactions.append([str(dataset.values[i,j]) for j in [0, len(dataset.columns) - 1]])

In [21]:
transactions

[['InvoiceNo', 'Country'],
 ['536365', 'United Kingdom'],
 ['536365', 'United Kingdom'],
 ['536365', 'United Kingdom'],
 ['536365', 'United Kingdom'],
 ['536365', 'United Kingdom'],
 ['536365', 'United Kingdom'],
 ['536365', 'United Kingdom'],
 ['536366', 'United Kingdom'],
 ['536366', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536367', 'United Kingdom'],
 ['536368', 'United Kingdom'],
 ['536368', 'United Kingdom'],
 ['536368', 'United Kingdom'],
 ['536368', 'United Kingdom'],
 ['536369', 'United Kingdom'],
 ['536370', 'France'],
 ['536370', 'France'],
 ['536370', 'France'],
 ['536370', 'France'],
 ['536370', 'France'],
 ['536370', 'France'],
 ['536370', 'France'],
 ['536

In [28]:
import numpy as np
"""
Класс инициируется 3мя параметрами:
- min_supp - минимальный support  который мы рассматриваем для ItemSet. Рассчитывается как % от количества транзакций
- max_items - максимальное количество елементов в нашем ItemSet
- min_items - минимальное количество элементов ItemSet
"""
class Eclat:
    #инициализация объекта класса
    def __init__(self, min_support = 0.01, max_items = 5, min_items = 2):
        self.min_support = min_support
        self.max_items = max_items
        self.min_items = min_items
        self.item_lst = list()
        self.item_len = 0
        self.item_dict = dict()
        self.final_dict = dict()
        self.data_size = 0
    
    #создание словаря из ненулевых объектов из всех транзакций (вертикальный датасет)
    def read_data(self, dataset):
        for index, row in dataset.iterrows():
            row_wo_na = row.dropna().unique()
            for item in row_wo_na:
                item = str(item).strip()
                if item in self.item_dict:
                    self.item_dict[item][0] += 1
                else:
                    self.item_dict.setdefault(item, []).append(1)
                self.item_dict[item].append(index)
        #задаем переменные экземпляра (instance variables)
        self.data_size = dataset.shape[0]
        self.item_lst = list(self.item_dict.keys())
        self.item_len = len(self.item_lst)
        self.min_support = self.min_support * self.data_size
        #print ("min_supp", self.min_support)
        
    #рекурсивный метод для поиска всех ItemSet по алгоритму Eclat
    #структура данных: {Item: [Supp number, tid1, tid2, tid3, ...]}
    def recur_eclat(self, item_name, tids_array, minsupp, num_items, k_start):
        if tids_array[0] >= minsupp and num_items <= self.max_items:
            for k in range(k_start+1, self.item_len):
                if self.item_dict[self.item_lst[k]][0] >= minsupp:
                    new_item = item_name + " | " + self.item_lst[k]
                    new_tids = np.intersect1d(tids_array[1:], self.item_dict[self.item_lst[k]][1:])
                    new_tids_size = new_tids.size
                    new_tids = np.insert(new_tids, 0, new_tids_size)
                    if new_tids_size >= minsupp:
                        if num_items >= self.min_items: self.final_dict.update({new_item: new_tids})
                        self.recur_eclat(new_item, new_tids, minsupp, num_items+1, k)
    
    #последовательный вызов функций определенных выше
    def fit(self, dataset):
        i = 0
        self.read_data(dataset)
        for w in self.item_lst:
            self.recur_eclat(w, self.item_dict[w], self.min_support, 2, i)
            i+=1
        return self
        
    #вывод в форме словаря {ItemSet: support(ItemSet)}
    def transform(self):
        return {k: "{0:.4f}%".format((v[0]+0.0)/self.data_size*100) for k, v in self.final_dict.items()}

In [29]:
#создадим экземпляр класса с нужными нам параметрами
model = Eclat(min_support = 0.01, max_items = 3, min_items = 3)

In [ ]:
#обучим
model.fit(dataset)

In [ ]:
#и визуализируем результаты
model.transform()

Преимущества перед алгоритмом Apriori:

* Требования к памяти: поскольку алгоритм ECLAT использует метод поиска в глубину, он использует меньше памяти, чем алгоритм Apriori.
* Скорость: алгоритм ECLAT обычно быстрее, чем алгоритм Apriori.
* Количество вычислений: алгоритм ECLAT не предусматривает повторного сканирования данных для вычисления отдельных значений поддержки.